In [1]:
from pathlib import Path
import pandas as pd

# 데이터셋 확인

In [2]:
dataset_dir = Path('/datasets/nia/230927')
annotations_dir = dataset_dir / 'annotations'
collections_dir = dataset_dir / 'collections'

### 확장자 확인

In [3]:
def get_all_suffixes(directory):
    suffixes = set()
    for p in Path(directory).rglob('*'):
        if len(p.suffix) > 0:
            suffixes.add(p.suffix)
    return suffixes

In [4]:
collections_suffixes = sorted(get_all_suffixes(collections_dir))
annotations_suffixes = sorted(get_all_suffixes(annotations_dir))
print('확장자')
print('데이터', collections_suffixes)
print('레이블', annotations_suffixes)

확장자
데이터 ['.csv', '.pcd', '.png']
레이블 ['.json']


### 채널 확인

In [8]:
def get_all_channels(directory):
    channels = set()
    for p in Path(directory).rglob('*.*'):
        channels.add(p.parts[-2])
    return channels

In [9]:
collections_channels = sorted(get_all_channels(collections_dir))
annotations_channels = sorted(get_all_channels(annotations_dir))
print('채널')
print('데이터', collections_channels)
print('레이블', annotations_channels)

채널
데이터 ['gps', 'image_B', 'image_F', 'image_L', 'image_R', 'lidar', 'thermal']
레이블 ['imageCaption', 'image_B', 'image_F', 'image_L', 'image_R', 'lidar', 'thermal']


### 채널-확장자 페어 확인

In [10]:
def get_all_channel_suffix_pairs(directory):
    pairs = set()
    for p in Path(directory).rglob('*.*'):
        pairs.add((p.parts[-2], p.suffix))
    return pairs

In [11]:
collections_pairs = get_all_channel_suffix_pairs(collections_dir)
annotations_pairs = get_all_channel_suffix_pairs(annotations_dir)

In [12]:
print('확장자-채널 페어')
for suffix in collections_suffixes:
    matches = sorted([pair[0] for pair in collections_pairs if pair[1] == suffix])
    print('데이터', f'{suffix}: {matches}')

for suffix in annotations_suffixes:
    matches = sorted([pair[0] for pair in annotations_pairs if pair[1] == suffix])
    print('레이블', f'{suffix}: {matches}')

확장자-채널 페어
데이터 .csv: ['gps']
데이터 .pcd: ['lidar']
데이터 .png: ['image_B', 'image_F', 'image_L', 'image_R', 'thermal']
레이블 .json: ['imageCaption', 'image_B', 'image_F', 'image_L', 'image_R', 'lidar', 'thermal']


# 디렉토리 구조 확인

### 가시광, 열영상 이미지

In [13]:
image_sample_path = next(collections_dir.rglob('*/image_F/*.png'))
label_sample_path = next(annotations_dir.rglob('*/image_F/*.json'))
print('이미지', image_sample_path.relative_to(dataset_dir))
print('레이블', label_sample_path.relative_to(dataset_dir))

이미지 collections/230823/230823_183210_K/230823_183210_K(3)/image_F/CK_A11_R04_erh_rainy_01005548_F.png
레이블 annotations/230823/230823_183210_K/230823_183210_K(3)/image_F/CK_A11_R04_erh_rainy_01005600_F.png.json


### 라이다 포인트 클라우드

In [14]:
image_sample_path = next(collections_dir.rglob('*/lidar/*.pcd'))
label_sample_path = next(annotations_dir.rglob('*/lidar/*.json'))
print('라이다', image_sample_path.relative_to(dataset_dir))
print('레이블', label_sample_path.relative_to(dataset_dir))

라이다 collections/230823/230823_183210_K/230823_183210_K(3)/lidar/LK_A11_R04_erh_rainy_01005640.pcd
레이블 annotations/230823/230823_183210_K/230823_183210_K(3)/lidar/LK_A11_R04_erh_rainy_01005614.json


### 구조 형태

In [12]:
file_format = '{date}/{date}_{time}_K/{date}_{time}_K({index})/{channel}/{filename}.{suffix}'
file_format_with_datetime = '{YYMMDD}/{YYMMDD}_{HHMMSS}_K/{YYMMDD}_{HHMMSS}_K({index})/{channel}/{filename}.{suffix}'
print(file_format)
print(file_format_with_datetime)

{date}/{date}_{time}_K/{date}_{time}_K({index})/{channel}/{filename}.{suffix}
{YYMMDD}/{YYMMDD}_{HHMMSS}_K/{YYMMDD}_{HHMMSS}_K({index})/{channel}/{filename}.{suffix}


### 데이터 수집 날짜 태그

In [13]:
dates = [p.name for p in collections_dir.iterdir()]
print('데이터 수집 날짜 태그', dates)

데이터 수집 날짜 태그 ['230822', '230823', '230906_1_modify']


### 데이터-레이블 구조 매칭 확인

In [14]:
def get_all_subdirectories_above_channels_in_relative_to(directory):
    subdirs = set()
    for p in Path(directory).rglob('*.*'):
        subdirs.add(p.parent.parent.relative_to(directory))
    return subdirs

In [15]:
collections_subdirs = get_all_subdirectories_above_channels_in_relative_to(collections_dir)
annotations_subdirs = get_all_subdirectories_above_channels_in_relative_to(annotations_dir)

In [16]:
print('데이터 서브디렉토리 수', len(collections_subdirs))
print('레이블 서브디렉토리 수', len(annotations_subdirs))
print('데이터 + 레이블 합집합 수', len(collections_subdirs & annotations_subdirs))
print('데이터 - 레이블 차집합 수', len(collections_subdirs - annotations_subdirs))
print('레이블 - 데이터 차집합 수', len(annotations_subdirs - collections_subdirs))

데이터 서브디렉토리 수 111
레이블 서브디렉토리 수 132
데이터 + 레이블 합집합 수 91
데이터 - 레이블 차집합 수 20
레이블 - 데이터 차집합 수 41


### 결론
어딘가 데이터, 레이블 구조 간의 차이가 존재한다.

In [17]:
print('데이터 - 레이블 차집합')
collections_subdirs - annotations_subdirs

데이터 - 레이블 차집합


{PosixPath('230906_1_modify/230906_095031_K'),
 PosixPath('230906_1_modify/230906_100857_K'),
 PosixPath('230906_1_modify/230906_101218_K'),
 PosixPath('230906_1_modify/230906_102055_K'),
 PosixPath('230906_1_modify/230906_102320_K'),
 PosixPath('230906_1_modify/230906_102507_K'),
 PosixPath('230906_1_modify/230906_103007_K'),
 PosixPath('230906_1_modify/230906_103143_K'),
 PosixPath('230906_1_modify/230906_103419_K'),
 PosixPath('230906_1_modify/230906_110706_K'),
 PosixPath('230906_1_modify/230906_110752_K'),
 PosixPath('230906_1_modify/230906_111433_K'),
 PosixPath('230906_1_modify/230906_113409_K'),
 PosixPath('230906_1_modify/230906_122254_K'),
 PosixPath('230906_1_modify/230906_122853_K'),
 PosixPath('230906_1_modify/230906_123652_K'),
 PosixPath('230906_1_modify/230906_123855_K'),
 PosixPath('230906_1_modify/230906_123940_K'),
 PosixPath('230906_1_modify/230906_124121_K'),
 PosixPath('230906_1_modify/230906_124330_K')}

In [18]:
print('레이블 - 데이터 차집합')
annotations_subdirs - collections_subdirs

레이블 - 데이터 차집합


{PosixPath('230906_1_modify/230906_095031_K/230906_095031_K(1)'),
 PosixPath('230906_1_modify/230906_100857_K/230906_100857_K(1)'),
 PosixPath('230906_1_modify/230906_100857_K/230906_100857_K(2)'),
 PosixPath('230906_1_modify/230906_100857_K/230906_100857_K(5)'),
 PosixPath('230906_1_modify/230906_100857_K/230906_100857_K(6)'),
 PosixPath('230906_1_modify/230906_101218_K/230906_101218_K(1)'),
 PosixPath('230906_1_modify/230906_101218_K/230906_101218_K(2)'),
 PosixPath('230906_1_modify/230906_102055_K/230906_102055_K(1)'),
 PosixPath('230906_1_modify/230906_102055_K/230906_102055_K(2)'),
 PosixPath('230906_1_modify/230906_102055_K/230906_102055_K(3)'),
 PosixPath('230906_1_modify/230906_102320_K/230906_102320_K(1)'),
 PosixPath('230906_1_modify/230906_102507_K/230906_102507_K(3)'),
 PosixPath('230906_1_modify/230906_103007_K/230906_103007_K(1)'),
 PosixPath('230906_1_modify/230906_103007_K/230906_103007_K(3)'),
 PosixPath('230906_1_modify/230906_103007_K/230906_103007_K(4)'),
 PosixPath

### 결론

230906_1_modify/230916_{time}_K/ 하위 디렉토리 매칭이 안된다.

기본 구조 형태에서 {date}_{time}_K({index}) 부분이 사라졌다.

collections에서 위 구조 형태가 빠져있고, annotations에는 위 구조 형태를 유지한다.

In [19]:
print('데이터 - 레이블 차집합 수', len(collections_subdirs - annotations_subdirs))
print('레이블 - 데이터 차집합 상위 디렉토리 수', len({p.parent for p in (annotations_subdirs - collections_subdirs)}))
print('데이터/230906_1_modify 하위 디렉토리 수', len(list((collections_dir / '230906_1_modify').iterdir())))

데이터 - 레이블 차집합 수 20
레이블 - 데이터 차집합 상위 디렉토리 수 20
데이터/230906_1_modify 하위 디렉토리 수 20


In [20]:
print('Q) 사라진 부분만 제외하면 상위 폴더 구조는 동일한가?')
print('A)', {p.parent for p in (annotations_subdirs - collections_subdirs)} == (collections_subdirs - annotations_subdirs))

Q) 사라진 부분만 제외하면 상위 폴더 구조는 동일한가?
A) True


### 결론
230906_1_modify 하위 폴더들은 이미지-레이블 페어 여부를 확실히 파악해야 한다.

## 이미지-레이블 페어 확인

In [21]:
print('Q) 사라진 부분만 제외하면 하위 폴더 구조는 동일한가?')
print('이미지', next(collections_dir.rglob('230906_1_modify/**/image_F/*.png')))
print('레이블', next(annotations_dir.rglob('230906_1_modify/**/image_F/*.json')))
print('A) 데이터의 파일명이 매칭되지 않는다.')

Q) 사라진 부분만 제외하면 하위 폴더 구조는 동일한가?
이미지 dataset/collections/230906_1_modify/230906_095031_K/image_F/CF_230906_095031_001.png
레이블 dataset/annotations/230906_1_modify/230906_095031_K/230906_095031_K(1)/image_F/CK_A01_R01_day_clear_01008462_F.png.json
A) 데이터의 파일명이 매칭되지 않는다.


In [22]:
print('Q) 최소한 데이터, 레이블의 숫자는 같은가?')
print('전방 가시광 이미지 수', len(list(collections_dir.rglob('230906_1_modify/**/image_F/*.png'))))
print('전방 가시광 레이블 수', len(list(annotations_dir.rglob('230906_1_modify/**/image_F/*.json'))))
print('A) 숫자마저도 다르다.')

Q) 최소한 데이터, 레이블의 숫자는 같은가?


전방 가시광 이미지 수 3600
전방 가시광 레이블 수 3444
A) 숫자마저도 다르다.


### 결론

230906_1_modify 하위 디렉토리 파일들의 경우 어떤 이미지와 레이블이 매칭되는지 모른다.

매칭 여부를 모르므로 현재 시점에선 학습, 추론에 사용이 불가능하다.

# 데이터-레이블 페어 확인

In [23]:
def is_there_paired_label_for_image(path):
    relative_path = Path(path).relative_to(collections_dir)
    paired_annotation_path = Path(str(annotations_dir / relative_path) + '.json')
    if paired_annotation_path.exists():
        return True
    else:
        return False


In [24]:
def is_there_paired_label_for_lidar(path):
    relative_path = Path(path).relative_to(collections_dir)
    paired_annotation_path = annotations_dir / relative_path.with_suffix('.json')
    if paired_annotation_path.exists():
        return True
    else:
        return False

In [25]:
def find_unpaired_collection(path):
    unpaired = []
    collection_path = Path(path)
    
    for path in collection_path.rglob('*.png'):
        if not is_there_paired_label_for_image(path):
            unpaired.append(path)
    
    for path in collection_path.rglob('*.pcd'):
        if not is_there_paired_label_for_lidar(path):
            unpaired.append(path)

    return unpaired

In [27]:
unpaired = {}
for date in dates:
    unpaired[date] = find_unpaired_collection(collections_dir / date)

In [28]:
print('데이터-레이블 페어 확인')
for date in dates:
    print('날짜', date, '매칭 실패 데이터 개수', len(unpaired[date]))

데이터-레이블 페어 확인
날짜 230822 매칭 실패 데이터 개수 0
날짜 230823 매칭 실패 데이터 개수 0
날짜 230906_1_modify 매칭 실패 데이터 개수 21600
